In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import numpy as np
import matplotlib.pyplot as plt
import kuaizi as kz
from astropy.table import Table
import astropy.units as u

from kuaizi.display import display_single
from astropy.io import fits
import os
import sys
from shutil import copyfile

In [20]:
kz.utils.set_env(project='HSC', name='LSBG', data_dir='/scratch/gpfs/jiaxuanl/Data/')

'/scratch/gpfs/jiaxuanl/Data/HSC/LSBG'

In [23]:
cat = Table.read('./Cutout/NSA/nsa_cutout_cat.fits')

In [24]:
cat

ra,dec,a_image,b_image,theta_image,ellipticity,mag_auto_g,mag_auto_r,mag_auto_i,flux_radius_ave_g,flux_radius_ave_i,mu_ave_g,mu_ave_i,acorr_ratio,g-i,g-r,A_g,A_r,A_i,tract,patch,synth_id,viz-id,is_candy,is_galaxy,is_tidal,is_outskirts,is_cirrus,is_junk,num_votes,good_votes,bad_votes,cutout_size,prefix,radius,image_flag [5],psf_flag [5]
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,arcmin,,
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,bytes3,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,bytes57,float64,bool,bool
233.8016874,43.5542719,6.492,5.061,5.95,0.22,23.5222,23.6272,22.9757,2.935632,0.8137080000000001,27.040179908948534,24.15471611513387,1.7362257902378089,1.1240313297700208,0.4055601455840791,0.06568911818117336,0.04654926376525461,0.033220447951196545,16004,"4,2",-1,18809,0,0,0,0,0,2,2,0,2,1.0,/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Cutout/NSA/nsa_18809,1.0,True .. True,True .. True
224.4561174,-0.9291037,14.119,9.869,53.11,0.301,19.6318,19.169,18.9775,2.69388,2.4884160000000004,23.14438259811236,22.36824095728265,1.2908956146130863,0.5915190410240171,0.42324076135458144,0.15190033815353532,0.1076410995081192,0.07681937917755342,9377,"0,3",-1,87312,1,0,0,0,0,0,1,1,0,1.0,/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Cutout/NSA/nsa_87312,1.0,True .. True,True .. True
341.3537285,3.2736835,7.361,5.714,12.23,0.224,22.3655,21.5069,21.1127,2.7330240000000003,3.6174600000000003,25.67505341732977,24.487508652348964,1.5925034031509135,0.9076084327185318,0.5710845704761327,0.22313681916207126,0.1581213896382014,0.11284525188060207,10183,"0,2",-1,89859,0,0,0,0,0,1,1,0,1,1.0,/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Cutout/NSA/nsa_89859,1.0,True .. True,True .. True
217.7986521,1.2353161,10.92,8.207,-37.26,0.248,22.0973,21.3385,21.0301,3.43938,5.1767520000000005,26.171154129076257,25.844846028114173,1.25510924806672,0.6874064533973304,0.4951610006885372,0.10721410273243694,0.07597510342097526,0.05422055612976628,9616,"4,7",-1,27799,2,0,0,0,0,0,2,2,0,1.0,/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Cutout/NSA/nsa_27799,1.0,True .. True,True .. True
180.7597942,-1.4974917,13.672,9.265,-12.76,0.322,20.496,20.055,19.8495,2.611644,2.3632560000000002,24.035064662514774,23.17199364778578,1.3162279669493444,0.6281640763069939,0.4342510387241492,0.06400434374185174,0.04535538246600134,0.03236842004884862,9106,"8,8",-1,29750,0,2,0,0,0,0,2,2,0,1.0,/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Cutout/NSA/nsa_29750,1.0,True .. True,True .. True
34.7036257,-6.7654166,12.291,11.173,26.74,0.091,20.8427,20.2532,20.0851,3.561936,4.286268,25.024618338828635,24.633132649198014,1.1445992724031469,0.5681974125559849,0.432182204397834,0.07385035369618563,0.05233255809401833,0.037347766252169345,8282,"2,4",-1,48372,2,0,0,0,0,0,2,2,0,1.0,/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Cutout/NSA/nsa_48372,1.0,True .. True,True .. True
351.1385074,1.5299615,12.815,10.12,-19.92,0.21,19.7768,19.3266,19.0715,2.735964,2.6691000000000003,23.45754237600703,22.702363492234216,1.230077458514783,0.6521869486901707,0.4116800410927032,0.10543272520943547,0.0747127663021394,0.05331967389960625,9948,"4,0",-1,26739,0,2,0,0,0,0,2,2,0,1.0,/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Cutout/NSA/nsa_26739,1.0,True .. True,True .. True
176.7385685,-1.9880663,12.92,8.987,-0.98,0.304,19.8322,19.031,18.6303,2.5476360000000002,2.3357040000000002,23.38033399579325,21.930052293729368,1.227850443205251,1.1453909109734925,0.785296394328556,0.05423891415688321,0.03843530848543749,0.027429825130375514,9103,"6,5",-1,65197,0,2,0,0,0,0,2,2,0,1.0,/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Cutout/NSA/nsa_65197,1.0,True .. True,True .. True


### Generate RGB images

In [25]:
def _display_cutout_rgb(images, ax=None):
    import scarlet
    from scarlet.display import AsinhMapping

    if ax is None:
        fig, ax = plt.subplots(figsize=(5, 5))
        
    # Display
    norm = AsinhMapping(minimum=-0.2, stretch=1, Q=1)
    img_rgb = scarlet.display.img_to_rgb(images, norm=norm)
    plt.imshow(img_rgb, origin='lower')
    ax.axis('off')
    
    if ax is None:
        return fig
    return ax

In [26]:
CUTOUT_DIR = '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Cutout/NSA/'
FIGURE_DIR = '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/'
WEBPAGE_DIR = '/tigress/jiaxuanl/public_html/NSA/cutout_RGB/figure/'

In [27]:
# Check file exsistence
for filt in 'grizy':
    for obj in cat:
        if not os.path.isfile(CUTOUT_DIR + f"nsa_{obj['viz-id']}_{filt}.fits"):
            print(f"NSA/nsa_{obj['viz-id']}_{filt}.fits not found!")

In [28]:
# Display each cutout
for i, obj in enumerate(cat):
    ind = obj['viz-id']
    
    images = np.asarray([fits.open(f'{CUTOUT_DIR}/nsa_{ind}_{filt}.fits')[1].data for filt in 'grizy'])
    fig, ax = plt.subplots(figsize=(5, 5))
    ax = _display_cutout_rgb(images, ax=ax)
    plt.savefig(FIGURE_DIR + f'nsa_{ind}_cutout.png', dpi=70, bbox_inches='tight')
    plt.close()
    if i % 10 == 0:
        print(f'Progress: {i} / {len(cat)}')

Progress: 0 / 100
Progress: 10 / 100
Progress: 20 / 100
Progress: 30 / 100
Progress: 40 / 100
Progress: 50 / 100
Progress: 60 / 100
Progress: 70 / 100
Progress: 80 / 100
Progress: 90 / 100


In [29]:
# Copy figures in FIGURE_DIR to WEBPAGE_DIR
for i, obj in enumerate(cat):
    ind = obj['viz-id']
    copyfile(FIGURE_DIR + f'nsa_{ind}_cutout.png', WEBPAGE_DIR + f'nsa_{ind}_cutout.png')

### Generate webpage for cutouts. WE use `diezi/webpage/gen_web_cutout.py`

In [30]:
sys.path.append('/home/jiaxuanl/Research/Packages/kuaizi/diezi/webpage/')
from gen_web_cutout import webpage_cutout

In [31]:
#FOR NSA SAMPLE
FIGURE_DIR = '/tigress/jiaxuanl/public_html/NSA/cutout_RGB/figure'
WEBPAGE_DIR = '/tigress/jiaxuanl/public_html/NSA/cutout_RGB/'
SCARLET_DIR = '/tigress/jiaxuanl/public_html/NSA/scarlet_result/'
sample_name = 'NSA'

In [32]:
webpage_cutout(FIGURE_DIR, WEBPAGE_DIR, sample_name, SCARLET_DIR=SCARLET_DIR, 
               col_num=6, row_num=4)

You have 98 galaxies to be displayed
Total pages: 5


In [33]:
print('The webpages are in', WEBPAGE_DIR)

The webpages are in /tigress/jiaxuanl/public_html/NSA/cutout_RGB/


### The search box on webpage still doesn't work. Need to work on the javascript file